# 股票新闻资讯爬取（基础ver）

lxml 4.6.3  
requests 2.24.0  
aip 4.15.5  
matplotlib 3.2.1

In [1]:
# !pip install lxml

In [2]:
# !pip install requests

In [3]:
# !pip install selenium

In [ ]:
import pymysql

In [28]:
import requests  # 导入必要的库
import matplotlib.pyplot as plt  # 导入必要的库
import pandas as pd  # 导入必要的库
from lxml import etree  # 导入必要的库
from selenium import webdriver  # 导入必要的库

In [29]:
import time  # 导入必要的库
import random  # 导入必要的库

东方财富资讯网址格式：https://guba.eastmoney.com/list,002422,1,f.html  
第二页：https://guba.eastmoney.com/list,002422,1,f_2.html 

In [ ]:
# 连接数据库
def connect_to_database(host, user, password, database, retries=5, delay=5):
    connection = None
    while not connection and retries > 0:
        try:
            connection = pymysql.connect(host=host,
                                         user=user,
                                         password=password,
                                         database=database,
                                         charset='utf8mb4',
                                         cursorclass=pymysql.cursors.DictCursor)
            print("已成功连接到数据库")
        except Exception as e:
            print(f"连接失败：{e}")
            retries -= 1
            print(f"重试连接，剩余尝试次数：{retries}")
            if retries > 0:
                time.sleep(delay)
    return connection

In [ ]:
# 向数据库插入数据
def insert_data(connection, table_name, data):
    with connection.cursor() as cursor:
        # Insert data into temporary table
        keys = ', '.join(data.keys())
        values = ', '.join(['%s'] * len(data))
        sql = f"INSERT INTO {table_name} ({keys}) VALUES ({values})"
        cursor.execute(sql, tuple(data.values()))
    connection.commit()

In [30]:
def download_news(codes):
concat_container = []
for code in codes:  # 循环遍历数据项
print(code)
url = "https://guba.eastmoney.com/list," + str(code) + ",1,f.html"
concat_container.append(parse_pages(url, code)) # 将每个个股的资讯拼接在一起  # 向列表添加数据
time.sleep(random.uniform(60, 70))  # 循环遍历数据项
concat_df = pd.concat(concat_container, ignore_index = True)
return concat_df

In [31]:
# 解析每个页面的数据
def parse_pages(url, code):
max_page = 3 # 最多截取 50 页的信息，用的时候可以更改； 此处用 3 页，获取最近一个季度的资讯；
concat_container = []
for i in range(1, max_page + 1):  # 循环遍历数据项
if i != 1:  # 条件判断
url = "https://guba.eastmoney.com/list," + str(code) + ",1,f_" + str(i) + ".html"
concat_container.append(download_page(url, code))  # 向列表添加数据
# time.sleep(random.uniform(1, 3))
concat_df = pd.concat(concat_container, ignore_index = True) # 将单个个股的每页资讯拼接在一起
return concat_df

In [8]:
headers = {'User-Agent':
'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.0.0 Safari/537.36'}

In [17]:
url = 'https://guba.eastmoney.com/list,002422,1,f.html'
page_data = requests.get(url, headers=headers).content.decode("utf-8")  # 使用requests库发送GET请求获取网页内容
data_tree = etree.HTML(page_data)

titles = data_tree.xpath('//*[@id="mainlist"]/div/ul/li[1]/table/tbody/tr/td[3]/div/a/text()')
time = data_tree.xpath('//*[@id="mainlist"]/div/ul/li[1]/table/tbody/tr/td[5]/div')
reading_numb = data_tree.xpath('//*[@id="mainlist"]/div/ul/li[1]/table/tbody/tr/td[1]/div')
comment_numb = data_tree.xpath('//*[@id="mainlist"]/div/ul/li[1]/table/tbody/tr/td[2]/div')
author = data_tree.xpath('//*[@id="mainlist"]/div/ul/li[1]/table/tbody/tr/td[4]/div/a/text()')
time_list = list(map(lambda x: x.text, time))
reading_numb_list = list(map(lambda x: x.text, reading_numb))
comment_numb_list = list(map(lambda x: x.text, comment_numb))
stock_code = ['002422' for i in range(len(time))]  # 循环遍历数据项

In [20]:
df_content = {
'stock_code': stock_code,
'release_time': time_list,
'news_titles': titles,
'reading_numb': reading_numb_list,
'comment_numb': comment_numb_list,
'author': author
}
pd.DataFrame(df_content)  # 使用pandas创建DataFrame

,stock_code,release_time,news_titles,reading_numb,comment_numb,author
0,002422,04-01 08:56,科伦药业：融资净买入464.61万元，融资余额7.29亿元（03-29）,84,0,科伦药业资讯
1,002422,03-31 03:10,科伦药业连续2周融资净偿还合计1.21亿元,206,0,大眼看两融
2,002422,03-31 02:15,深股通连续2周净卖出科伦药业 累计净卖出8250.78万元,161,0,胖猫周周见
3,002422,03-29 08:48,科伦药业：融资净买入384.55万元，融资余额7.24亿元（03-28）,228,0,科伦药业资讯
4,002422,03-28 08:51,科伦药业：融资余额7.2亿元，创近一年新低（03-27）,288,1,科伦药业资讯
...,...,...,...,...,...,...
75,002422,02-23 08:52,科伦药业：连续3日融资净买入累计3288.68万元（02-22）,197,0,科伦药业资讯
76,002422,02-22 08:59,科伦药业：融资净买入1576.41万元，融资余额8.1亿元（02-21）,208,0,科伦药业资讯
77,002422,02-21 08:49,科伦药业：融资净买入114.98万元，融资余额7.94亿元（02-20）,171,0,科伦药业资讯
78,002422,02-20 08:56,科伦药业：融资余额7.93亿元，创近一年新低（02-19）,251,0,科伦药业资讯


In [32]:
# 解析指定页面的数据并保存至本地
def download_page(url, code):
try:  # 异常处理，尝试执行代码块，处理潜在的错误
headers = {'User-Agent':
'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.0.0 Safari/537.36'}####根据情况改动
page_data = requests.get(url, headers=headers).content.decode("utf-8")  # 使用requests库发送GET请求获取网页内容
data_tree = etree.HTML(page_data)
titles = data_tree.xpath('//*[@id="mainlist"]/div/ul/li[1]/table/tbody/tr/td[3]/div/a/text()')
time_raw = data_tree.xpath('//*[@id="mainlist"]/div/ul/li[1]/table/tbody/tr/td[5]/div')
reading_numb = data_tree.xpath('//*[@id="mainlist"]/div/ul/li[1]/table/tbody/tr/td[1]/div')
comment_numb = data_tree.xpath('//*[@id="mainlist"]/div/ul/li[1]/table/tbody/tr/td[2]/div')
author = data_tree.xpath('//*[@id="mainlist"]/div/ul/li[1]/table/tbody/tr/td[4]/div/a/text()')
time_list = list(map(lambda x: x.text, time_raw))
reading_numb_list = list(map(lambda x: x.text, reading_numb))
comment_numb_list = list(map(lambda x: x.text, comment_numb))
stock_code = [ str(code) for i in range(len(time_raw))]  # 循环遍历数据项
# save_path = '/Users/wujiewang/Desktop/stocks/' ##### 用的时候改为自己的本地文件夹地址
# file_name = save_path + str(code) + ".txt" # 文件保存路径，确保这个文件夹存在
# for title in titles:
#     title = title + "\r\n"
#     with open(file_name, "a") as file:
#         file.write(title)
df_content = {
'stock_code': stock_code,
'release_time': time_list,
'news_titles': titles,
'reading_numb': reading_numb_list,
'comment_numb': comment_numb_list,
'author': author
}
download_sheet = pd.DataFrame(df_content)  # 使用pandas创建DataFrame
return download_sheet
except:  # 异常处理，尝试执行代码块，处理潜在的错误
print("服务器超时")

In [33]:
codes = ['000568',
'000596',
'000617',
'000625',
] # 测试4只股票是否成功。后续改为上证 300
test0 = download_news(codes)
test0

000568
000596
000617
000625


,stock_code,release_time,news_titles,reading_numb,comment_numb,author
0,000568,04-01 09:33,泸州老窖4月1日快速上涨,77,0,泸州老窖资讯
1,000568,04-01 08:52,泸州老窖：连续4日融资净偿还累计2627.19万元（03-29）,60,0,泸州老窖资讯
2,000568,03-31 03:06,泸州老窖本周融资净买入3001.78万元，居酿酒行业板块第二,107,0,大眼看两融
3,000568,03-31 02:07,泸州老窖本周深股通持股市值增加1.08亿元，居酿酒行业板块第五,110,0,胖猫周周见
4,000568,03-29 08:45,泸州老窖：连续3日融资净偿还累计1756.28万元（03-28）,108,0,泸州老窖资讯
...,...,...,...,...,...,...
955,000625,02-06 01:27,长安汽车1月销量超28万辆 同比增长63.19%,1566,10,长安汽车资讯
956,000625,02-06 12:45,同比增长63.19% 1月长安汽车销量达28.04万辆,731,0,长安汽车资讯
957,000625,02-06 12:34,1月份长安汽车销量280367辆 同比增63.19%,629,0,长安汽车资讯
958,000625,02-06 12:14,午间公告：长安汽车1月汽车销量同比增加63.19%,3150,6,长安汽车资讯


In [154]:
codes = ['000568',
'000596',
'000617',
'000625',
'000651',
'000661',
'000708',
'000725',
'000733',
'000768',
'000776'] # 测试三只股票是否成功。后续改为上证 300
test = download_news(codes)

000568
000596
000617
000625
000651
000661
000708
000725
000733
000768
000776


bug 记录：超过 15 个公司会自动触发反爬虫，因此加入时间休息。  
如何发现：二分法逼近问题所在

In [155]:
test

,stock_code,release_time,news_titles
0,000568,03-29 08:45,泸州老窖：连续3日融资净偿还累计1756.28万元（03-28）
1,000568,03-29 07:28,野村东方国际证券维持泸州老窖增持评级 目标价260元
2,000568,03-28 05:43,泸州老窖3月28日大宗交易成交999.32万元
3,000568,03-28 04:44,大宗交易：机构账户卖出泸州老窖999.32万元（03-28）
4,000568,03-28 03:48,国信证券维持泸州老窖买入评级 预计2023年净利润同比增长27%
...,...,...,...
2635,000776,11-12 03:07,广发证券本周融资净买入4370.56万元，居证券板块第十五
2636,000776,11-12 02:08,广发证券本周获深股通增持313.24万股 最新持股量8436万股
2637,000776,11-11 07:44,广发证券11月10日被深股通减持32.98万股
2638,000776,11-10 08:44,广发证券：连续3日融资净偿还累计4315.66万元（11-09）


In [126]:
test[test['stock_code'] == '002422'] # 测试单个股票是否成功

,stock_code,release_time,news_titles
0,002422,03-29 08:48,科伦药业：融资净买入384.55万元，融资余额7.24亿元（03-28）
1,002422,03-28 08:51,科伦药业：融资余额7.2亿元，创近一年新低（03-27）
2,002422,03-27 04:19,科伦药业：“科伦转债”赎回登记日为3月29日
3,002422,03-27 08:53,科伦药业：融资净偿还669.07万元，融资余额7.41亿元（03-26）
4,002422,03-26 08:44,科伦药业：融资净买入1679.46万元，融资余额7.47亿元（03-25）
...,...,...,...
235,002422,11-19 05:16,深股通连续4周净卖出科伦药业 累计净卖出5.51亿元
236,002422,11-17 05:58,科伦药业：11月17日召开董事会会议
237,002422,11-17 08:48,科伦药业：连续4日融资净买入累计1902.47万元（11-16）
238,002422,11-16 10:20,科伦药业副总经理、董秘冯昊：上市公司做好合规治理最需要加强透明度


In [34]:
stock_info = pd.read_csv('/Users/wujiewang/Desktop/QF5214/inf_stock_data.csv')
stock_info.head()

,Unnamed: 0,EXCH_ENG,SEC_NAME,TRADE_CODE,CONCEPT,HOTCONCEPT,LATESTCONCEPT,CHAIN,COMP_NAME,COMP_NAME_ENG,PROVINCE,NATURE1,INDUSTRY_CSRC2023,INDUSTRY_CSRCCODE2023,INDUSTRY_GICS,INDUSTRY_GICSCODE
0,000001.SZ,SZSE,平安银行,1,跨境支付;银;大盘股;上涨点位贡献;全A等权;银行精选;全A精选;陆股通重仓前100;核心资...,跨境支付;银;上涨点位贡献;银行精选;核心资产;基金重仓,中动量,NaN,平安银行股份有限公司,"Ping An Bank Co., Ltd.",广东省,公众企业,金融业,J,金融,40
1,000002.SZ,SZSE,万科A,2,全A负动量;房地产金股;海通证券金股;券商金股综合;银;大盘股;下跌点位贡献;全A等权;房地...,银;房地产精选;核心资产;分拆上市;基金重仓;新型城镇化,全A负动量,PC 长租公寓,万科企业股份有限公司,"China Vanke Co.,Ltd.",广东省,公众企业,房地产业,K,房地产,60
2,000063.SZ,SZSE,中兴通讯,63,英伟达产业链;6G;新经济;自主可控;TMT;信创产业;通信金股;券商金股综合;长城证券金股...,英伟达产业链;6G;自主可控;信创产业;EDA;核心资产;新基建;5G应用;消费电子产业;半...,全A价格回报,"PC 家庭娱乐,PC 在线办公,PC NCP专利,PC 智能网联汽车,PC 物联网,PC 5...",中兴通讯股份有限公司,Zte Corporation,广东省,公众企业,制造业,C,信息技术,45
3,000069.SZ,SZSE,华侨城A,69,央企房地产;央企中盘股;非陆股通重仓;国企综合;央企综合;中盘股;银;全A等权;全A(非陆股...,国企综合;银;业绩爆雷;社保重仓,央企房地产,NaN,深圳华侨城股份有限公司,"Shenzhen Overseas Chinese Town Co., Ltd",广东省,中央国有企业,房地产业,K,房地产,60
4,000100.SZ,SZSE,TCL科技,100,非基金重仓;新经济;TMT;元宇宙;先进制造;大盘股;下跌点位贡献;三新;双循环;全A等权;...,元宇宙;先进制造;双循环;5G应用;消费电子产业;新型显示技术;分拆上市;科技龙头;数字经济...,非基金重仓,"PC 家庭娱乐,PC 智能手表,PC 消费金融,PC 家电",TCL科技集团股份有限公司,Tcl Technology Group Corporation,广东省,公众企业,制造业,C,信息技术,45


In [35]:
hold_stocks = list(stock_info.iloc[:, 0].apply(lambda x: x[:6]))
hold_stocks[:5]

['000001', '000002', '000063', '000069', '000100']

result = download_news(hold_stocks)

result[result['stock_code'] == '000568']

result[result['stock_code'] == '600958']
超过 15 个就触发反爬虫

In [167]:
# 进行第二次测试，测试 20 个
result2 = download_news(hold_stocks[:20])

000001
000002
000063
000069
000100
000157
000166
000301
000333
000338
000408
000425
000538
000568
000596
000617
000625
000651
000661
000708


In [168]:
result2

,stock_code,release_time,news_titles
0,000001,03-29 08:41,平安银行：融资净偿还5438.84万元，两市排名第16（03-28）
1,000001,03-28 08:47,平安银行：融资净偿还6432.45万元，两市排名第18（03-27）
2,000001,03-27 06:06,平安银行：平安金积存业务起投金额由500元上调至600元
3,000001,03-27 05:14,平安银行金积存业务起投金额由500元上调至600元
4,000001,03-27 08:43,平安银行：融资净买入2633.03万元，融资余额52.02亿元（03-26）
...,...,...,...
4795,000708,03-04 03:24,没有地板问题不大，继续观望持有，目前还有几个点盈利[哈欠]
4796,000708,03-04 03:21,下7.7就重仓，，你跌吧，，，
4797,000708,03-04 03:20,快买买买买
4798,000708,03-04 02:35,网传蚂蚁把大象吃了，公司也不辟谣！大象有49%是谁的？[坏笑]


In [174]:
result2[result2['stock_code'] == '000425']

,stock_code,release_time,news_titles
2640,000425,03-29 08:41,徐工机械：融资净偿还2369.31万元，融资余额8.73亿元（03-28）
2641,000425,03-29 04:00,徐工机械03月28日获深股通增持419.86万股
2642,000425,03-28 08:52,徐工机械：融资净买入565.29万元，融资余额8.97亿元（03-27）
2643,000425,03-27 08:45,徐工机械：融资净买入345.67万元，融资余额8.91亿元（03-26）
2644,000425,03-26 08:43,徐工机械：融资净偿还5401.54万元，创近一年新高（03-25）
...,...,...,...
2875,000425,11-05 03:07,徐工机械本周融资净买入1.31亿元，居工程机械板块第一
2876,000425,11-05 02:08,徐工机械本周深股通持股市值减少1.46亿元，居工程机械板块第二
2877,000425,11-04 07:51,徐工机械11月03日获深股通增持413.29万股
2878,000425,11-03 03:18,徐工机械等8000万元成立电气科技公司


 遇到 bug：result2中，超过 12 个股票就不行。考虑是一段时间内，可访问数量有限制。尝试在每个股票之间休息 60s。

In [182]:
# 进行第三次测试，测试 20 个
result3 = download_news(hold_stocks[:20])

000001
000002
000063
000069
000100
000157
000166
000301
000333
000338
000408
000425
000538
000568
000596
000617
000625
000651
000661
000708


In [ ]:
# 输出爬取的表格
result.to_csv('/Users/wujiewang/Desktop/QF5214/stock_news')  # 将DataFrame保存到CSV文件，方便后续处理

判断是否被反爬虫机制识别并采取了措施通常可以通过监控你的爬虫程序的行为和接收到的响应来实现。下面是一些常见的指标和反爬虫形式：

判断被反爬虫的指标
访问速度突然变慢：如果你发现爬虫访问特定网站的速度突然变得非常慢，这可能是网站对你的请求速度进行了限制。

返回状态码：常见的有403 Forbidden或429 Too Many Requests。这些状态码表明你的请求被服务器拒绝或因请求过多而被限制。

页面内容改变：如果返回的页面内容突然改变，比如返回了一个验证码页面或者提示“请确认你不是机器人”的消息，这也是被识别为爬虫的迹象。

IP被封禁：如果你的爬虫突然无法访问目标网站，而你使用其他网络（如手机数据）可以访问，这可能意味着你的IP地址被封禁。

被重定向到登录或验证码页面：有些网站会在检测到爬虫行为时重定向请求到登录页或验证码验证页，以阻止进一步爬取。

常见的反爬虫形式
用户代理检查（User-Agent）：网站可能会检查HTTP请求的User-Agent头，如果识别为常见的爬虫或脚本工具，则拒绝访问。

IP地址限制：网站可能会限制特定IP地址在一定时间内的请求次数，超过限制可能会导致临时或永久封禁。

验证码验证：一些网站在检测到异常流量时会要求用户输入验证码，这对自动化爬虫是一个挑战。

动态令牌/JavaScript挑战：一些网站会使用JavaScript生成动态令牌或进行某些计算验证，以防止没有执行JavaScript的爬虫访问。

HTTP Referer检查：检查请求的来源页，如果没有或不符合预期，请求可能会被拒绝。

Cookies和会话验证：一些网站要求请求携带有效的Cookies或进行会话跟踪，缺少这些可能会被视为爬虫。

应对策略  
减慢请求速度：增加请求之间的延时，模拟人类用户的行为。  
更换用户代理：定期更换用户代理字符串，避免使用默认或爬虫明显的用户代理。  
使用代理IP：使用代理服务器或IP池来避免IP封禁。  
处理验证码：对于简单验证码，可以尝试使用OCR（光学字符识别）技术自动解决。对于复杂情况，可能需要人工干预或使用第三方服务。  
执行JavaScript：使用像Selenium这样能够执行JavaScript的工具来获取完整渲染后的页面内容。  
尽管有这些策略可以尝试，但最重要的是遵守目标网站的爬虫政策，尊重网站的robots.txt文件，并确保你的爬虫行为不会对网站造成负担或损害。  

In [184]:
result3 # 第三次测试成功，说明间隔 1 分钟成立。

,stock_code,release_time,news_titles
0,000001,03-29 08:41,平安银行：融资净偿还5438.84万元，两市排名第16（03-28）
1,000001,03-28 08:47,平安银行：融资净偿还6432.45万元，两市排名第18（03-27）
2,000001,03-27 06:06,平安银行：平安金积存业务起投金额由500元上调至600元
3,000001,03-27 05:14,平安银行金积存业务起投金额由500元上调至600元
4,000001,03-27 08:43,平安银行：融资净买入2633.03万元，融资余额52.02亿元（03-26）
...,...,...,...
4795,000708,10-29 10:56,华泰证券维持中信特钢买入评级 目标价19.76元
4796,000708,10-28 07:54,中信特钢10月27日被深股通减持9.92万股
4797,000708,10-28 12:19,中信特钢：10月27日召开董事会会议
4798,000708,10-27 11:03,中信特钢：前三季度归母净利润43.74亿元 同比减少21.09%


In [37]:
# 进行正式提取
result4 = pd.DataFrame()  # 使用pandas创建DataFrame
container = []
for i in range(0, 300, 25):  # 循环遍历数据项
result4_i = download_news(hold_stocks[i:i+25])
if i == 0:  # 条件判断
result4 = result4_i
else:  # 条件判断
container = [result4, result4_i]
result4 = pd.concat(container, ignore_index = True)

000001
000002
000063
000069
000100
000157
000166
000301
000333
000338
000408
000425
000538
000568
000596
000617
000625
000651
000661
000708
000725
000733
000768
000776
000786
000792
000800
000858
000876
000877
000895
000938
000963
000977
000983
000999
001289
001979
002001
002007
002027
002049
002050
002074
002129
002142
002179
002180
002202
002230
002236
002241
002252
002271
002304
002311
002352
002371
002410
002415
002459
002460
002466
002475
002493
002555
002594
002601
002603
002648
002709
002714
002736
002812
002821
002841
002916
002920
002938
003816
300014
300015
300033
300059
300122
300124
300142
300223
300274
300308
300316
300347
300408
300413
300433
300450
300454
300496
300498
300628
300661
300750
300751
300759
300760
300763
300782
300896
300919
300957
300979
300999
301269
600000
600009
600010
600011
600015
600016
600018
600019
600023
600025
600028
600029
600030
600031
600036
600039
600048
600050
600061
600085
600089
600104
600111
600115
600132
600150
600176
600183
600188
600196

In [38]:
result4

,stock_code,release_time,news_titles,reading_numb,comment_numb,author
0,000001,04-01 08:51,平安银行：融资净偿还9308.54万元，两市排名第15（03-29）,98,0,平安银行资讯
1,000001,03-31 03:07,平安银行本周融资净偿还1.71亿元，两市排名第13,137,0,大眼看两融
2,000001,03-31 02:11,平安银行本周深股通持股市值增加1.89亿元，居银行板块第五,191,1,胖猫周周见
3,000001,03-29 04:54,平安银行：截至2023年末 本行共有在职员工43119人（含派遣人员、含理财子和分行）,2802,10,平安银行资讯
4,000001,03-29 02:48,平安发布《2023年可持续发展报告》 ESG评级保持国内领先,896,3,平安银行资讯
...,...,...,...,...,...,...
71995,688981,11-14 01:57,光大证券维持中芯国际买入评级 预计2023年净利润同比减少50.33%,192,0,公司研报提示
71996,688981,11-14 07:45,中芯国际：连续6日融资净买入累计3.21亿元（11-13）,221,0,中芯国际资讯
71997,688981,11-14 07:44,华金证券给予中芯国际买入评级，代工龙头营收持续增长，设备提前进场加速产能扩张,2749,19,中芯国际资讯
71998,688981,11-14 07:04,【调研快报】中芯国际接待华泰证券等多家机构调研,1870,1,中芯国际资讯


In [40]:
result4.to_csv('/Users/wujiewang/Desktop/QF5214/stock_news_titles_like_comments')  # 将DataFrame保存到CSV文件，方便后续处理

In [39]:
result4.to_csv('/Users/wujiewang/Desktop/QF5214/stock_news_titles_like_comments.csv', encoding='utf_8_sig', index=False)  # 将DataFrame保存到CSV文件，方便后续处理

In [ ]:
# 要操作的表名
table_name = "ods_news_ef_stock"

In [ ]:
# 登录数据库需要的信息
host='47.***.***.45',
database='qf5214',
user='user',
password='********'

In [ ]:
connection = connect_to_database(host, user, password, database) # 创建数据库连接

In [ ]:
insert_data(connection, table_name, result4)

## 测试功能：提取出最大页数

这一块⬇️代码还未跑通

后续改进空间：  
1、设计一个动态提取函数，用于提取每个股票的最大页数，减少资源浪费

In [67]:
# 获取url下的最大page数
def get_max_page(url):
page_data = requests.get(url).content.decode("utf-8")  # 使用requests库发送GET请求获取网页内容
data_tree = etree.HTML(page_data)
if page_data.find("page_newslib"):  # 条件判断
max_page = data_tree.xpath("//*[@class=\"page_newslib\"]//a[last()-1]/text()")
return int(max_page[0])
else:  # 条件判断
return 1

In [60]:
# # 设置Selenium浏览器驱动（这里以Chrome为例）
# driver = webdriver.Chrome(executable_path='/Users/wujiewang/Downloads/chrome-mac-arm64/chromedriver')
# # 目标网页URL
# url = 'https://guba.eastmoney.com/list,002422,1,f.html'
# # 使用Selenium打开网页
# driver.get(url)
# # 等待一定时间，让页面加载完成，可能需要根据网速和网页的复杂度调整等待时间
# time.sleep(5)  # 等待5秒
# # 获取页面源代码
# page_data = driver.page_source
# # 解析页面源代码
# data_tree = etree.HTML(page_data)
# # 使用XPath表达式提取所需信息
# element = data_tree.xpath('//*[@id="mainlist"]/div/ul/li[1]/ul/li[7]/a/text()')
# # 输出提取的内容
# print(element)
# # 关闭浏览器
# driver.quit()

In [72]:
url = 'https://guba.eastmoney.com/list,002422,1,f.html'
page_data = requests.get(url, headers=headers).content.decode("utf-8")  # 使用requests库发送GET请求获取网页内容
data_tree = etree.HTML(page_data)
title = data_tree.xpath('//*[@id="mainlist"]/div/ul/li[1]/table/tbody/tr/td[3]/div/a/text()')